In [265]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.matlib import repmat

from timeit import default_timer as timer

import tqdm

In [61]:
class FunctionCompilation():
    # feval equivalent
    def callFunction(self, name, val):
        fn = getattr(self, name, None)
        if fn is not None:
            return fn(val)
        else:
            print('Undefined function call')
            return
        
        
    def ackley(self, val):
        dim = val.shape[1]
        
        f1 = np.sum([val[:, :1 + dim]**2], axis=-1)
        f1 = np.reshape(f1, (f1.shape[1], f1.shape[0]))
        f2 = np.sum([np.cos(2*np.pi * val[:, :1+dim])], axis=-1)
        f2 = np.reshape(f2, (f2.shape[1], f2.shape[0]))
        
        return -20 * np.exp(-0.2 * np.sqrt(1/dim * f1)) - np.exp(1/dim * f2) + 20 + np.exp(1)

In [62]:
class DEParams(object):
    
    def __init__(self, I_itermax=5e5, I_NP=50, F_weight=0.5, F_CR=0.9, I_bnd_constr=3):
        
        ## Set parameters for HyDE-DF
        self.I_itermax = I_itermax
        self.I_NP = I_NP
        self.F_weight = F_weight
        self.F_CR = F_CR
        
        self.I_bnd_constr = I_bnd_constr #Using bound constraints is possible to change direct in DE
        # 1 repair to the lower or upper violated bound
        # 2 rand value in the allowed range
        # 3 bounce back
        
        
class OtherParameters(object):
    
    def __init__(self, objfun, dim, lowerlimit, upperlimit):
        self.objfun = objfun
        self.dim = dim
        self.lowerlimit = lowerlimit
        self.upperlimit = upperlimit

In [247]:
def HyDE_DF(deParameters, otherParameters, initialSolution):
    
    # Generate population
    def genpop(a, b, lowMatrix, upMatrix):
        return np.random.uniform(low=lowMatrix, high=upMatrix, size=(a, b))
    
    
    # Trial generation
    def generate_trial(F_weight, F_CR, FM_pop, FVr_bestmemit, I_NP, I_D, FVr_rot, linear_decrease):
    
        # Save the old population
        FM_popold = FM_pop

        # Index pointer array
        FVr_ind = np.random.permutation(np.arange(5))

        # Shuffle locations of vectors
        FVr_a1 = np.random.permutation(np.arange(I_NP))

        # Rotate indices by ind[0] positions
        FVr_rt = (FVr_rot + FVr_ind[0]) % I_NP

        # Rotate vector locations
        FVr_a2 = FVr_a1[FVr_rt]

        # Shuffled population 1
        FM_pm1 = FM_popold[FVr_a1, :]

        # Shuffled population 2
        FM_pm2 = FM_popold[FVr_a2, :]


        # Meaning the same F_CR for all individuals
        if len(F_CR) == 1:

            # All random numbers < F_CR are 1, 0 otherwise
            FM_mui = (np.random.normal(size=(I_NP, I_D)) < F_CR).astype(int)

            # Inverse mask to FM_mui
            FM_mpo = (FM_mui < 0.5).astype(int)

        # Meaning a different F_CR for each individual
        else:
            # All random numbers < F_CR are 1, 0 otherwise
            FM_mui = (np.random.normal(size=(I_NP, I_D)) < repmat(F_CR, 1, I_D)).astype(int)

            # Inverse mask to FM_mui
            FM_mpo = FM_mui < 0.5


        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 
        FM_bm = repmat(FVr_bestmemit, I_NP, 1)

        # Linear decrease
        a = linear_decrease

        # Exponential decreasing function
        ginv = np.exp(1-(1/a**2)) 


        #differential variation
        repmat0 = np.reshape(repmat(F_weight[:, 2], 1, I_D), (F_weight.shape[0], I_D))
        repmat1 = np.reshape(repmat(F_weight[:, 0], 1, I_D), (F_weight.shape[0], I_D))
        repmat2 = np.reshape(repmat(F_weight[:, 1], 1, I_D), (F_weight.shape[0], I_D))
        
        diff_var = ginv * (repmat1 * (FM_bm * (repmat2 + np.random.normal(size=(I_NP, I_D)) - FM_popold)))

        FM_ui = FM_popold + repmat0 * (FM_pm1 - FM_pm2) + diff_var

        FM_ui = FM_popold * FM_mpo + FM_ui * FM_mui
        FM_base = FM_bm
        #msg = 'HyDE-DF'
        #print(msg)

        return FM_ui, FM_base, None
    
    
    # Update aux function        
    def _update(p, lowMatrix, upMatrix, BRM, FM_base):
        #print(p.shape)
        if BRM == 1: # Our method
            # [popsize, dim] = size(p)
            idx = np.where(p < lowMatrix)
            p[idx[0], idx[1]] = lowMatrix[idx]

            idx = np.where(p > upMatrix)
            p[idx] = upMatrix[idx]
        elif BRM == 2: # Random initialization - DOES NOT WORK
            idx = [np.where(p < lowMatrix), np.where(p > upMatrix)]
            replace = np.random.uniform(low=lowMatrix[idx[0][0], idx[0][1]], 
                                        high=upMatrix[idx[1][0], idx[1][1]], 
                                        size=(len(idx), 1))
            p[idx] = replace
        elif BRM == 3: # Bounce-back
            idx = np.where(p < lowMatrix)
            new_val = np.random.uniform(low=lowMatrix[idx[0], idx[1]], 
                                        high=FM_base[idx[0], idx[1]], 
                                        size=(len(idx[0]), len(idx[1])))
            if new_val.shape[0] > 0:
                p[idx[0], idx[1]] = new_val[:, 0]
                
                
            idx = np.where(p > upMatrix)
            new_val = np.random.uniform(low=FM_base[idx[0], idx[1]], 
                                        high=upMatrix[idx[0], idx[1]], 
                                        size=(len(idx[0]), len(idx[1])))
            if new_val.shape[0] > 0:
                p[idx[0], idx[1]] = new_val[:, 0]

        return p
    
    
    #-----This is just for notational convenience and to keep the code uncluttered.--------
    I_NP = deParameters.I_NP
    F_weight = deParameters.F_weight
    F_CR = deParameters.F_CR
    I_D = otherParameters.dim     #Number of variables or dimension
    deParameters.nVariables = I_D
    FVr_minbound = otherParameters.lowerlimit * np.array(np.ones((1,otherParameters.dim)))
    FVr_maxbound = otherParameters.upperlimit * np.array(np.ones((1,otherParameters.dim)))
    I_itermax = deParameters.I_itermax
    
    
    #Repair boundary method employed
    BRM = deParameters.I_bnd_constr   #1: bring the value to bound violated
                                      #2: repair in the allowed range
                                      #3: Bounce-back
            
    # Get Objective Function
    fnc= otherParameters.objfun
    
    # Function caller initialization
    fn_obj = FunctionCompilation()
    
    #-----Check input variables---------------------------------------------
    if I_NP < 5:
        I_NP = 5
        print('I_NP increased to minimal value 5\n')
    
    if (F_CR < 0) | (F_CR > 1):
        F_CR = 0.5
        print('F_CR should be from interval [0, 1] - set to default value 0.5\n')
        
    if I_itermax <= 0:
        I_itermax = 200
        print('I_itermax should be > 0 - set to default value 200\n')
    
    
    #-----Initialize population and some arrays-------------------------------
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # pre-allocation of loop variables
    fitMaxVector = np.empty((1, I_itermax))
    fitMaxVector[:] = np.nan
    
    
    # limit iterations by threshold
    gen = 0; #iterations
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #----FM_pop is a matrix of size I_NPx(I_D+1). It will be initialized------
    #----with random values between the min and max values of the-------------
    #----parameters-----------------------------------------------------------
    # FLC modification - vectorization
    minPositionsMatrix = repmat(FVr_minbound, I_NP, 1)
    maxPositionsMatrix = repmat(FVr_maxbound, I_NP, 1)
    deParameters.minPositionsMatrix = minPositionsMatrix
    deParameters.maxPositionsMatrix = maxPositionsMatrix
    
    # generate initial population.
    FM_pop = genpop(I_NP,I_D,minPositionsMatrix,maxPositionsMatrix)
    
    #If you want to inject initial solutions
    if initialSolution != None: 
        noInitialSolutions = initialSolution.shape[0]
        FM_pop[1:noInitialSolutions, :] = initialSolution
        
        
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #------Evaluate the best member after initialization----------------------
    # Modified by FLC
    S_val = fn_obj.callFunction(fnc, FM_pop)
    I_best_index = np.argmin(S_val); # This mean that the best individual correspond to the best worst performance
    FVr_bestmemit = FM_pop[I_best_index, :] # best member of current iteration
    
    fitMaxVector[:, 0] = S_val[I_best_index] #We save the mean value and mean penalty value
    # The user can decide to save the mean, best, or any other value here
    
    
    #------DE-Minimization------------------------------------------------
    #------FM_popold is the population which has to compete. It is--------
    #------static through one iteration. FM_pop is the newly--------------
    #------emerging population.-------------------------------------------
    FVr_rot  = np.arange(0, I_NP, 1)             # rotating index array (size I_NP)
    
    
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #% HYDE self-adaptive parameters
    F_weight_old = repmat(F_weight, I_NP, 3)
    F_weight = F_weight_old
    F_CR_old = repmat(F_CR, I_NP, 1)
    F_CR = F_CR_old

    
    while gen < I_itermax-1:  #%&&  fitIterationGap >= threshold
        #% Calculate decay function factor a = itr / MaxItr; 
        lin_decr = (I_itermax-gen) / I_itermax


        #% Update HyDE-DF values              
        value_R = np.random.normal(size=(I_NP,3))
        ind1 = (value_R < 0.1).astype(int)
        ind2 = (np.random.normal(size=(I_NP,1)) < 0.1).astype(int)
        
        F_weight[:] = F_weight_old[:]
        F_weight[ind1] = (0.1 + np.random.normal(size=(sum(sum(ind1)), 1)) * 0.9)[0]
        
        F_CR[:] = F_CR_old[:]
        F_CR[ind2] = np.random.normal(size=(sum(ind2)[0], 1))[0]
        
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        FM_ui, FM_base, _ = generate_trial(F_weight, F_CR, FM_pop, FVr_bestmemit, I_NP, I_D, FVr_rot, lin_decr)
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 
        ## Boundary Control
        FM_ui = _update(FM_ui,minPositionsMatrix,maxPositionsMatrix,BRM,FM_base)
        
        ## Evaluation of new Pop
        S_val_temp = fn_obj.callFunction(fnc, FM_ui)
        
        
        ## Elitist Selection
        ind = np.where(S_val_temp < S_val)
        S_val[ind] = S_val_temp[ind]
        FM_pop[ind, :] = FM_ui[ind, :]
        
        
        ## Update best results
        S_bestval = min(S_val)
        I_best_index = np.argmin(S_val)
        FVr_bestmemit = FM_pop[I_best_index, :]
        
        
        ## Save best parameters (similar to jDE)
        F_weight_old[ind, :] = F_weight[ind, :]
        F_CR_old[ind] = F_CR[ind]
        
        gen += 1;        
        
        ## Store fitness evolution and obj fun evolution as well
        fitMaxVector[:, gen] = S_bestval
        
        
    Fit_and_p = fitMaxVector[0, gen-1]
    
    return Fit_and_p, FVr_bestmemit, fitMaxVector

In [268]:
N_RUNS = 30

deParams = DEParams(I_itermax=1000)
otherParams = OtherParameters(objfun='ackley', lowerlimit=-32, upperlimit=32, dim=30)

list_fit_and_p = []
list_bestmemit = []
list_maxvector = []
list_timer = []


print('Starting trials')
for i in tqdm.tqdm(np.arange(N_RUNS)):
    
    # Start timer - https://stackoverflow.com/questions/7370801/how-to-measure-elapsed-time-in-python
    start = timer()
    
    # Do optimization
    fit_and_p, bestmemit, maxvector = HyDE_DF(deParams, otherParams, None)
    
    # End timer
    end = timer()
    
    list_fit_and_p.append(fit_and_p)
    list_bestmemit.append(bestmemit)
    list_maxvector.append(maxvector)
    list_timer.append(end-start)
    

trial_results = pd.DataFrame({'Fit': list_fit_and_p,
                              'Solution': list_bestmemit,
                              'Fit Vector': list_maxvector,
                              'Time': list_timer})

Starting trials


100%|██████████████████████████████████████████████████████| 30/30 [00:32<00:00,  1.07s/it]


In [270]:
# Show solutions

trial_results

,Fit,Solution,Fit Vector,Time
0,2.725682,"[-0.27419260340692453, 0.6415251570727177, 0.4...","[[20.794072702508984, 20.784094749721433, 20.7...",0.881067
1,3.349452,"[0.7401271991681752, 0.00710893186259437, 0.46...","[[20.65038755171325, 20.03493295955793, 20.034...",0.908553
2,3.706943,"[0.3452779381898745, -0.39673852641165486, -0....","[[20.54186676837528, 20.54186676837528, 20.541...",1.025852
3,2.803405,"[0.2801444740884434, 0.13446316905003708, 0.38...","[[20.76924097440192, 20.76924097440192, 20.132...",0.946554
4,3.535540,"[-0.03637350717174992, -0.0668753771474131, 0....","[[20.606820362084736, 20.336995556621602, 20.3...",0.902365
5,3.497225,"[-0.5027029247945212, 1.0881097729429512, 0.26...","[[20.667828718524685, 20.667828718524685, 20.6...",1.022966
6,4.328285,"[0.16235522742638187, -0.48448257210427564, -1...","[[20.386522591811275, 20.386522591811275, 20.3...",1.359705
7,3.366964,"[-0.05874980513575101, -0.6431206634486828, 0....","[[20.28759030883054, 20.28759030883054, 19.691...",1.067099
8,2.498555,"[0.4827000381748906, -0.07954841623005482, -0....","[[20.582099342362003, 20.582099342362003, 20.2...",1.063185
9,3.532167,"[-0.3415502423875226, -0.021814676838318725, 0...","[[20.532104446688127, 20.485600266065777, 19.7...",0.856098
